In [1]:
import numpy as np
import socket
from PIL import Image
from keras.models import load_model
%matplotlib inline

width = 64
height = 64
channel = 3

Using TensorFlow backend.


In [2]:
lidar_model = load_model('car_model_lidar_adecuate.h5')
camera_model = load_model('car_model_camera_adecuate.h5')

In [3]:
class EnvironmentInteraction:
    
    def __init__(self, Address, Port):
        
        self.byteDelimeter = b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4
        self.address = Address
        self.port = Port
        self.sock = socket.socket()
        self.sock.bind((self.address, self.port))
            
        
    def listen(self):
        
        self.sock.listen(1)
        self.connection, self.address = self.sock.accept()
        
        
    def step(self, action):
        
        self.connection.send(action.encode('utf-8'))                        # Send action to environment
        data = self.connection.recv(768432)                                 # Get camera and lidar images as byte array
        
        data = data.split(self.byteDelimeter)
        
        camera = Image.frombytes('RGB', (64, 64), data[0])
        lidar = Image.frombytes('RGB', (64, 64), data[1])
                
        return np.array(camera.getdata(), dtype = np.float32).reshape(1, width, height, channel) / 255, np.array(lidar.getdata(), dtype = np.float32).reshape(1, width, height, channel) / 255
    
environment = EnvironmentInteraction('', 7777) 

In [4]:
lidar_target_names = ['Human', 'Clear path', 'Obstacle']
lane_target_names = ['Left Lane', 'Right Lane']
lastDetected = ''
counter = 0
obstacleDetected = False

def command(lane_output, angle_output, lidar_output):
    
    global lidar_target_names
    global lane_target_names
    global counter
    global obstacleDetected
    
    
    action = '0'
    
    if counter == 15:
        counter = 0
        obstacleDetected = False
    
    if ((obstacleDetected) & (lidar_target_names[lidar_output] != 'Obstacle')):
        counter += 1
        return '0'
    
    if lidar_target_names[lidar_output] == 'Human':
        action = '-999'     # Stop moving
    elif lidar_target_names[lidar_output] == 'Obstacle':
        obstacleDetected = True
        if lane_target_names[lane_output] == 'Left Lane':
            action = '45'   # move to right lane
        else:
            action = '-45'  # move to left lane
    else:
        action = str(angle_output)
    
    return action

def AngleRange(angle):
    if angle > 45:
        return 45
    elif angle < -45:
        return -45
    else:
        return angle

In [12]:
environment.listen()
print(environment.connection)

action = '0#_#_'

while(True):
        
    cameraImage, lidarImage = environment.step(action)
    
    cameraImage = np.flip(cameraImage[0], axis = 0)    
    lidarImage = np.flip(lidarImage[0], axis = 0)
    
    camera_output = camera_model.predict(np.array(cameraImage).reshape(1, width, height, channel))
    lidar_output = lidar_model.predict(np.array(lidarImage).reshape(1, width, height, channel))
    
    angle = AngleRange((-1 * camera_output[1][0][0] + camera_output[1][0][1]) * 60)
    
    action = command(np.argmax(camera_output[0]), angle, np.argmax(lidar_output)) + '#' + lane_target_names[np.argmax(camera_output[0])] + '#' + lidar_target_names[np.argmax(lidar_output)]
    
    print(action)

<socket.socket fd=1044, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7777), raddr=('127.0.0.1', 50008)>
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0#Left Lane#Clear path
0.0

ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение